In [28]:
first_file_path = "../Fajlovi/1000G_test.vcf"
second_file_path = "../Fajlovi/1000G_test_2.vcf"

merged_file = open("merged.vcf","w+")

#možda da napravim jednu veliku klasu VCF_file u kojoj ću imati metode za sortiranje dictionaries-a filters i formats

class File_format: 
    def __init__(self, line):
        self.line = line
        self.vcf_version = ""
        self.extract_file_format_version()
        
    def __eq__(self, other):
        return self.line == other.line
    
    def extract_file_format_version(self):
        try:
            version = self.line.split("=VCFv",1)[1]
            self.vcf_version = version
        except:
            pass
    

class Filter_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.description = ""
        self.extract_id()
        self.extract_description()
        
    def extract_id(self):
        try:
            id = (self.line.split("ID="))[1].split(",")[0]
            self.id = id
        except:
            pass
    
    def extract_description(self):
        try:
            description = (self.line.split('Description="'))[1].split('"')[0]
            self.description = description
        except:
            pass

# ##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
class Format_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = (self.line.split('Description="'))[1].split('"')[0]
        except:
            pass
    
class Filter_lifted_variant:
    def __init__(self, line):
        self.line = line
        #da li nam trebaju sva ona pojedinačna polja
    
    
# ##INFO=<ID=GentrainScore,Number=.,Type=Float,Description="Gentrain Score">
class Info_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = (self.line.split('Description="'))[1].split('"')[0]
        except:
            pass
        
# ##contig=<ID=chr13,length=114364328,assembly=20>
class Contig_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.length = ""
        self.assembly = ""
        self.extract_id()
        self.extract_length()
        self.extract_assembly()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_length(self):
        try:
            self.number = (self.line.split("length="))[1].split(",")[0]            
        except:
            pass
        
    def extract_assembly(self):
        try:
            self.type = (self.line.split("assembly="))[1].split(">")[0]            
        except:
            pass

    
def write_filter_fields(filters):
    for key, value in sorted(filters.items()):
        merged_file.write(value.line)
    
    #do I need to close merged_file ???
    #get rid of filters, GarbageCollector or just filters = {}

def write_format_fields(formats):
    for key, value in sorted(formats.items()):
        merged_file.write(value.line)
        

#pošto se filteri trebaju sortirati trebam kupiti filtere u jednu listu 
#da li ovdje koristiti listu ili nešto drugo, uzećemo dictionary
#kod filtera sortiranje ide po ID 
file_formats = {}
filters = {}
formats = {}
infos = {}
contigs = {}
filter_lifted_variants = list()
filters_written = False
formats_written = False
filter_lifted_variants_written = False
infos_written = False
contigs_written = False

#WILDIDEA
#da li napraviti da je izlazni fajl kopija većeg fajla i onda dodavati dodatne linije
   
#ovo moramo napraiti da bude samo jedna funkcija, da može da radi i prvi i drugi fajl, a kasnije i proizvoljan niz fajlova
def read_specific_tag_first_file(tag, first_file_object, second_file_object):
    #first we read through first file, then second
    previous_position_first_file = first_file_object.tell()
    line_first_file = first_file_object.readline()
    
    while(line_first_file.startswith("##" + tag)):
        print("Read_specfic: " + str(first_file_object.tell()))
        create_object_add_to_dict(tag, line_first_file)
        previous_position_first_file = first_file_object.tell()
        line_first_file = first_file_object.readline()
    
    #rewind first file 
    first_file_object.seek(previous_position_first_file)
    read_specific_tag_second_file(tag, first_file_object, second_file_object)
    
def read_specific_tag_second_file(tag, first_file_object, second_file_object):
    #first we read through first file, then second
    previous_position_second_file = second_file_object.tell()
    line_second_file = second_file_object.readline()
    
    while(line_second_file.startswith("##" + tag)):
        create_object_add_to_dict(tag, line_first_file)
        previous_position_second_file = second_file_object.tell()
        line_second_file = second_file_object.readline()
    
    #we are done with the one tag, we need to write it in the merged_file
    
    #rewind second file 
    second_file_object.seek(previous_position_second_file) 
    write_tags_in_file(tag)
    
def write_tags_in_file(tag):
    if(tag == "fileformat"):
        # first sort 
        for key, value in sorted(file_formats.items()):
            merged_file.write(value.line)
    elif(tag == "FILTER"):
        for key, value in sorted(filters.items()):
            merged_file.write(value.line)
    elif(tag == "FORMAT"):
        for key, value in sorted(formats.items()):
            merged_file.write(value.line)
    elif(tag == "FilterLiftedVariants"):
        for value in filter_lifted_variants:
            merged_file.write(value.line) 
    elif(tag == "INFO"):
        for key, value in sorted(infos.items()):
            merged_file.write(value.line)
    elif(tag == "contig"):
        for key, value in sorted(contigs.items()):
            merged_file.write(value.line)
    

def create_object_add_to_dict(tag, line):
    if(tag == "fileformat"):
        file_format = File_format(line)
        file_formats[file_format.vcf_version] = file_format
    elif(tag == "FILTER"):
        filter_header = Filter_header(line)
        if(filter_header not in filters.values()):
            filters[filter_header.id] = filter_header
    elif(tag == "FORMAT"):
        format_header = Format_header(line)
        if(format_header not in formats.values()):
            formats[format_header.id] = format_header  
    elif(tag == "FilterLiftedVariants"):
        filter_lifted_variant = Filter_lifted_variant(line)
        filter_lifted_variants.append(filter_lifted_variant) 
    elif(tag == "INFO"):
        info_header = Info_header(line)
        if(info_header not in infos.values()):
            infos[info_header.id] = info_header
    elif(tag == "contig"):
        contig_header = Contig_header(line)
        if(contig_header not in contigs.values()):
            contigs[contig_header.id] = contig_header 

# context manager
with open(first_file_path) as first_file_object:
    with open(second_file_path) as second_file_object:
        #skontati kada prestati sa čitanjem fajlova
        line_first_file = first_file_object.readline()
        #if not line_first_file:
        #    break
        while(line_first_file.startswith("##")):
            print("Main position: " + str(first_file_object.tell()))
            tag = (line_first_file.split("##"))[1].split("=")[0] 
            create_object_add_to_dict(tag, line_first_file)
            read_specific_tag_first_file(tag, first_file_object, second_file_object)
            line_first_file = first_file_object.readline()
    

Main position: 22
Main position: 96
Read_specfic: 231
Read_specfic: 323
Read_specfic: 388
Read_specfic: 453
Read_specfic: 518
Read_specfic: 583
Read_specfic: 646
Read_specfic: 711
Read_specfic: 776
Read_specfic: 867
Main position: 933
Read_specfic: 995
Main position: 2073
Main position: 2136
Read_specfic: 2212
Read_specfic: 2289
Main position: 2336
Read_specfic: 2384
Read_specfic: 2432
Read_specfic: 2480
Read_specfic: 2528
Read_specfic: 2576
Read_specfic: 2624
Read_specfic: 2671
Read_specfic: 2718
Read_specfic: 2765
Read_specfic: 2812
Read_specfic: 2859
Read_specfic: 2906
Read_specfic: 2953
Read_specfic: 3000
Read_specfic: 3047
Read_specfic: 3094
Read_specfic: 3141
Read_specfic: 3188
Read_specfic: 3235
Read_specfic: 3282
Read_specfic: 3329
Read_specfic: 3382
Read_specfic: 3435
Read_specfic: 3488
Read_specfic: 3541
Read_specfic: 3594
Read_specfic: 3646
Read_specfic: 3698
Read_specfic: 3750
Read_specfic: 3803
Read_specfic: 3856
Read_specfic: 3908
Read_specfic: 3960
Read_specfic: 4012
Rea